# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("../output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,-1.85,98,99,3.58,RU,1715681198
1,1,grytviken,-54.2811,-36.5092,-7.51,76,54,2.65,GS,1715681198
2,2,akwanga,8.9167,8.3833,29.29,62,100,0.36,NG,1715681198
3,3,oriximina,-1.7656,-55.8661,23.56,97,100,0.96,BR,1715681198
4,4,enewetak,11.3474,162.3373,27.54,83,87,9.54,MH,1715681198


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points( x = 'Lng', y = 'Lat', size = 'Humidity', hover_cols = ['City'], geo = True, 
                                      tiles = 'CartoLight', title = 'City Humidity Map')
    

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df.loc[city_data_df['Humidity'] > 70]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dudinka,69.4058,86.1778,-1.85,98,99,3.58,RU,1715681198
1,1,grytviken,-54.2811,-36.5092,-7.51,76,54,2.65,GS,1715681198
3,3,oriximina,-1.7656,-55.8661,23.56,97,100,0.96,BR,1715681198
4,4,enewetak,11.3474,162.3373,27.54,83,87,9.54,MH,1715681198
6,6,invercargill,-46.4000,168.3500,8.90,82,41,1.79,NZ,1715681148
...,...,...,...,...,...,...,...,...,...,...
573,573,talara,-4.5772,-81.2719,20.91,78,64,6.55,PE,1715681333
575,575,oak hill,37.9723,-81.1487,14.71,85,100,2.03,US,1715681333
578,578,palmer,42.1584,-72.3287,12.34,79,100,2.57,US,1715681333
581,581,pontianak,-0.0333,109.3333,27.99,88,75,2.57,ID,1715681334


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,dudinka,RU,69.4058,86.1778,98,
1,grytviken,GS,-54.2811,-36.5092,76,
3,oriximina,BR,-1.7656,-55.8661,97,
4,enewetak,MH,11.3474,162.3373,83,
6,invercargill,NZ,-46.4000,168.3500,82,
...,...,...,...,...,...,...
573,talara,PE,-4.5772,-81.2719,78,
575,oak hill,US,37.9723,-81.1487,85,
578,palmer,US,42.1584,-72.3287,79,
581,pontianak,ID,-0.0333,109.3333,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "radius" : radius, 
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")


for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']

     # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] : f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = f"https://api.geoapify.com/v2/places"


     # Make and API request using the params dictionaty
    response = requests.get(base_url, params = params )

    
    # Convert the API response to JSON format
    name_address = response.json()


      # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
hotel_df


Starting hotel search
dudinka - nearest hotel: Муром
grytviken - nearest hotel: No hotel found
oriximina - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
invercargill - nearest hotel: The Grand
bethel - nearest hotel: Hotel Zero Degrees
adamstown - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
valle de la pascua - nearest hotel: Hotel El Triunfo
kem' - nearest hotel: Кузова
puerto penasco - nearest hotel: The Cove at Whale Hill
whitehorse - nearest hotel: Airport Chalet
novyy urgal - nearest hotel: Комнаты отдыха Новый Ургал
wailua homesteads - nearest hotel: Sheraton Kauai Coconut Beach Resort
blackmans bay - nearest hotel: Kingston Hotel
lincoln - nearest hotel: Holiday Inn Lincoln Southwest
waitangi - nearest hotel: Hotel Chathams
nemuro - nearest hotel: ホテルねむろ海陽亭
cockburn town - nearest hotel: The Salt Raker Inn
ares - nearest hotel: Hotel Casa Grande
puerto natales - nearest hotel: Hotel Hallef
vilyuchinsk - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
0,dudinka,RU,69.4058,86.1778,98,Муром
1,grytviken,GS,-54.2811,-36.5092,76,No hotel found
3,oriximina,BR,-1.7656,-55.8661,97,No hotel found
4,enewetak,MH,11.3474,162.3373,83,No hotel found
6,invercargill,NZ,-46.4000,168.3500,82,The Grand
...,...,...,...,...,...,...
573,talara,PE,-4.5772,-81.2719,78,Hotel Punta Pariñas
575,oak hill,US,37.9723,-81.1487,85,Comfort Inn New River
578,palmer,US,42.1584,-72.3287,79,No hotel found
581,pontianak,ID,-0.0333,109.3333,88,Airy Pontianak Selatan Imam Bonjol 89


In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points( x = 'Lng', y = 'Lat', size = 'Humidity', hover_cols = ['City', 'Hotel Name'], geo = True, 
                                      tiles = 'CartoLight', title = 'City Map with Hotels')
    

# Display the map
map_plot

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
hotel_df = hotel_df[hotel_df["Hotel Name"] != "No hotel found"]
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,dudinka,RU,69.4058,86.1778,98,Муром
6,invercargill,NZ,-46.4000,168.3500,82,The Grand
11,bethel,US,41.3712,-73.4140,87,Hotel Zero Degrees
14,valle de la pascua,VE,9.2155,-66.0073,77,Hotel El Triunfo
16,kem',RU,64.9555,34.5793,92,Кузова
...,...,...,...,...,...,...
572,zaozersk,RU,69.4005,32.4476,99,Северное сияние
573,talara,PE,-4.5772,-81.2719,78,Hotel Punta Pariñas
575,oak hill,US,37.9723,-81.1487,85,Comfort Inn New River
581,pontianak,ID,-0.0333,109.3333,88,Airy Pontianak Selatan Imam Bonjol 89


In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points( x = 'Lng', y = 'Lat', size = 'Humidity', hover_cols = ['City', 'Hotel Name'], geo = True, 
                                      tiles = 'CartoLight', title = 'City Map with Hotels')
    

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name)